In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:

# Пример данных
# Пожалуйста, замените этот пример вашими данными
data = [
    {"processes": ["Excel", "System", "Registry"], "isWorkMode": True},
    {"processes": ["Steam", "System", "Registry"], "isWorkMode": False},
    {"processes": ["Discord", "System", "Registry"], "isWorkMode": False},
    {"processes": ["Word", "System", "Registry"], "isWorkMode": True},
    {"processes": ["Word", "Discord", "System", "Registry"], "isWorkMode": True},
    # Добавьте больше данных
]


In [ ]:

# Подготовка данных
processes = []
labels = []

for entry in data:
    processes.append(' '.join(entry["processes"]))
    labels.append(entry["isWorkMode"])


In [ ]:

# Кодируем метки (True/False) в числа (1/0)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [ ]:

# Преобразование строк в числовые векторы
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(processes)
sequences = tokenizer.texts_to_sequences(processes)
X = tf.keras.preprocessing.sequence.pad_sequences(sequences)


In [ ]:

# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)


In [ ]:

# Создание модели
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))  # Входной слой
model.add(Dense(32, activation='relu'))  # Скрытый слой
model.add(Dense(1, activation='sigmoid'))  # Выходной слой


In [ ]:

# Компиляция модели
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)


In [ ]:

# Оценка модели
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')


In [ ]:

# Прогнозирование
def predict_is_work_mode(procs):
    seq = tokenizer.texts_to_sequences([procs])
    padded = tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen=X_train.shape[1])
    prediction = model.predict(padded)
    return prediction[0][0] > 0.5  # И возвращаем True/False


In [ ]:

# Пример использования
test_processes = ["proc1 proc2"]
print(f'Is Work Mode: {predict_is_work_mode(test_processes)}')
